In [ ]:
!pip install psycopg2-binary

In [ ]:
# from lifetimes.fitters import modified_beta_geo_fitter
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
#import numpy as np
# from lifetimes import BetaGeoFitter,GammaGammaFitter,ModifiedBetaGeoFitter

#from sklearn.metrics import mean_squared_error
#from math import sqrt
from sklearn.preprocessing import MinMaxScaler
#from datetime import date
import datetime
import psycopg2
from datetime import date



def start_new_customers_analysis():

        con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
        port= '5439', user= 'mantix', password= 'Mantix123!')
        cur = con.cursor()

        orders_data = pd.read_sql("select * from cdp.orders_data;",con)

        orders_data=orders_data[(orders_data['tenant_id']=='TNB00001') & (orders_data['store_id']=='STOM000000001')]

        orders_data = orders_data[~orders_data['status'].isin(['Abandoned','Errored','Pending','PendingReview','Cancelled','Null'])]

        items_data = pd.read_sql("select * from cdp.items_data;",con)


        orders_data.order_number = orders_data.order_number.astype(int).astype(str)
        items_data.order_number = items_data.order_number.astype(int).astype(str)
        

        ord_cust = orders_data[['customer_account_id','submitted_date','order_number']]
        items_df = items_data[['product_name','product_code','total','order_number','quantity','refund_amount']]
        items_df['refund_amount'].fillna(0,inplace=True)

        items_df=items_df.groupby(['order_number'])['total'].sum()-items_df.groupby(['order_number'])['refund_amount'].sum()        
        items_df=items_df.reset_index()
        items_df.columns=['order_number','total']


        df =pd.merge(ord_cust,items_df[['total','order_number']],on=['order_number'],how='inner')
        # df.to_csv('df_all.csv')
        print("%%%%%%%",df.shape)
        new_customers=df[df['customer_account_id'].map(df.customer_account_id.value_counts()==1)]
        print(new_customers.shape)
        new_customers['submitted_date']=pd.to_datetime(new_customers.submitted_date)
        print(new_customers.head())
        # new_customers['submitted_date']=new_customers['submitted_date'].dt.strftime('%d/%m/%Y')
        # new_customers.to_csv('new_customers.csv')
        today = datetime.date.today()
        print(today)
        first = today.replace(day=1)
        print(first)
        last_month = first - datetime.timedelta(days=1)
        new_customers=new_customers[new_customers['submitted_date']>=pd.to_datetime(last_month.replace(day=1))]
        new_customers=new_customers[new_customers['submitted_date']<pd.to_datetime(first)]
        new_customer_data=new_customers.groupby(['submitted_date'])['total'].sum()
        new_customer_data=pd.DataFrame(new_customer_data)
        new_customer_data.reset_index(inplace=True)
        new_customer_data['tenant_id'] = 'TNB00001'
        new_customer_data['store_id'] = 'STOM000000001'
        today_date = datetime.date.today()
        new_customer_data['create_dms'] = today_date

        # new_customer_data.to_csv('new_customer_Data.csv')
        print(new_customer_data.head())

        
        # new_customer_data.reset_index(inplace=True)
        # new_customer_data['tenant_id'] = 'TNB00001'
        # new_customer_data['store_id'] = 'STOM000000001'
        # today_date = date.today()
        # new_customer_data['create_dms'] = today_date
        # new_customer_data.to_csv('lkjhg.csv')
        # print(new_customer_data.head())


         

        return new_customer_data

def load_db(new_customer_data):
    # new_customer_data['InvoiceDate'] = pd.to_datetime(new_customer_data['InvoiceDate'],format='%Y%m%d')
    # print(new_customer_data)
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            #return 1

    def create_table(conn,table):
        try:
            query_col = """( InvoiceDate VARCHAR(1000),
                TotalPrice VARCHAR(1000),tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
            query = f"""
                    CREATE TABLE {table} 
                    """+query_col
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, insert_req):
        """ Execute a single INSERT request """
        cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
        
    def batch_insert(conn, batch_df):
        # Create a cursor object from the connection
        cursor = conn.cursor()
        statment=  f"""INSERT into {table} (InvoiceDate,TotalPrice,tenant_id,store_id,create_dms) values(%s,%s,%s,%s,%s);"""
        try:
            # Execute the batch INSERT request
            cursor.executemany(statment, batch_df)
            # Commit the changes to the database
            conn.commit()
            print("Batch INSERT successful.")
        except (Exception, psycopg2.DatabaseError) as error:
            # Rollback the transaction in case of an error
            conn.rollback()
            print("Batch INSERT failed, Error:", error)
        finally:
            # Close the cursor
            cursor.close() 
        
        
    def insert_dataframe(final_df,conn,table):
        batch_df=[]
        for i in final_df.index:
            current_tuple=(
                        str(new_customer_data.loc[i]['submitted_date'].date()), 
                        str(round(new_customer_data.loc[i]['total'],2)),
                        str(new_customer_data.loc[i]['tenant_id']),
                        str(new_customer_data.loc[i]['store_id']),
                        str(new_customer_data.loc[i]['create_dms'])
                )
            batch_df.append(current_tuple)
            if (len(batch_df) == 500): 
                batch_insert(conn, batch_df)
                batch_df=[]
                print("Current index: {}, time: {}".format(i, datetime.datetime.now()))
            #endif
        # Insert any remaining rows
        if (len(batch_df) > 0):
            batch_insert(conn,batch_df)
        print('Successfully inserted dataframe into the table')
        
    table = 'cdp.new_customers_value'
    drop_table(con,table)
    create_table(con,table)
    startTime = datetime.datetime.now()
    insert_dataframe(new_customer_data,con,table)
    print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

if __name__ == '__main__':

    # start cltv
    new_customer_data = start_new_customers_analysis()

    # load data into database
    load_db(new_customer_data)

In [ ]:
# def load_db(new_customer_data):
#     # new_customer_data['InvoiceDate'] = pd.to_datetime(new_customer_data['InvoiceDate'],format='%Y%m%d')
#     # print(new_customer_data)
#     con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
#     port= '5439', user= 'mantix', password= 'Mantix123!')
#     cur = con.cursor()

#     def drop_table(conn,table):
#         try:
#             query = f"""DROP TABLE IF EXISTS {table}"""
#             cur = conn.cursor()
#             cur.execute(query)
#             conn.commit()
#             print(f'Drop table {table} successfully')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             #return 1

#     def create_table(conn,table):
#         try:
#             query_col = """( InvoiceDate VARCHAR(1000),
#                 TotalPrice VARCHAR(1000),tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
#             query = f"""
#                     CREATE TABLE {table} 
#                     """+query_col
#             cur = conn.cursor()
#             cur.execute("ROLLBACK")
#             conn.commit()
#             cur.execute(query)
#             conn.commit()
#             print(f'Successfully created the table {table}')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             return 1
            
#     def single_insert(conn, insert_req):
#         """ Execute a single INSERT request """
#         cursor = conn.cursor()
#         try:
#             cursor.execute(insert_req)
#             conn.commit()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cursor.close()
#             return 1
#         cursor.close()
        
#     def insert_dataframe(new_customer_data,conn,table):
#         # new_customer_data['InvoiceDate']=new_customer_data['InvoiceDate'].astype('datetime64[us]')
#         # print(type(new_customer_data['InvoiceDate'][0]))
#         for i in new_customer_data.index:
#             print(new_customer_data.loc[i]['total'])
#             # print(new_customer_data.loc[i]['InvoiceDate'].date())
#             # print(type(new_customer_data.loc[i]['InvoiceDate']))
#             # new_customer_data.loc[i]['InvoiceDate']=new_customer_data.loc[i]['InvoiceDate'].to_pydatetime()
#             # print(type(new_customer_data.loc[i]['InvoiceDate']))
#             query = f"""
#                     INSERT into {table} (InvoiceDate,TotalPrice,tenant_id,store_id,create_dms) 
#                         values(%s,%s,%s,%s,%s);
#                     """ % (
                        
#                         "'"+str(new_customer_data.loc[i]['submitted_date'].date())+"'", 
#                         "'"+str(round(new_customer_data.loc[i]['total'],2))+"'",
                        
#                         # new_customer_data.loc[i]['cltv_12M'],
                        
#                         # new_customer_data.loc[i]['cltv_24M'],
                        
#                         "'"+str(new_customer_data.loc[i]['tenant_id'])+"'",
#                         "'"+str(new_customer_data.loc[i]['store_id'])+"'",
#                          "'"+str(new_customer_data.loc[i]['create_dms'])+"'")
#             if i%50==0:
#                 print(i,end=' ')
#             single_insert(conn, query)
#         conn.commit()
#         print('\nSuccessfully inserted dataframe into the table')
        
#     table = 'cdp.new_customers_value'
#     drop_table(con,table)
#     create_table(con,table)
#     insert_dataframe(new_customer_data,con,table)
